In [1]:
import pandas as pd
import datetime
import datetime
import re
import lxml
from bs4 import BeautifulSoup
import json
from calendar import monthrange
import numpy as np

In [2]:
!ls -lrth ../stayz/WebData/nsw_calendar/

total 1626520
-rw-r--r--  1 taj  staff    13M Mar  6 21:12 stayz_calendar_2018-03-04.json.zip
-rw-r--r--  1 taj  staff   5.7M Mar  6 21:18 stayz_calendar_2018-02-22a.json.zip
-rw-r--r--  1 taj  staff   594K Mar  6 21:18 stayz_calendar_2018-03-02.json.zip
-rw-r--r--  1 taj  staff    14M Mar  6 21:18 stayz_calendar_2018-02-26.json.zip
-rw-r--r--  1 taj  staff    13M Mar  6 21:18 stayz_calendar_2018-02-28.json.zip
-rw-r--r--  1 taj  staff   8.6M Mar  6 23:22 stayz_calendar_2018-03-03.json.zip
-rw-r--r--  1 taj  staff   3.8M Mar  9 09:04 stayz_calendar_2018-03-07.json.zip
-rw-r--r--  1 taj  staff    13M Mar  9 09:05 stayz_calendar_2018-03-08.json.zip
-rw-r--r--  1 taj  staff    14M Mar 14 08:55 stayz_calendar_2018-03-12.json.zip
-rw-r--r--  1 taj  staff    15M Mar 14 08:55 stayz_calendar_2018-03-13.json.zip
-rw-r--r--  1 taj  staff    13M Mar 15 21:21 stayz_calendar_2018-03-15.json.zip
-rw-r--r--  1 taj  staff   2.3M Mar 15 21:22 stayz_calendar_2018-03-14.json.zip
-rw-r--r--  1 taj  staff 

In [3]:
date_str = datetime.datetime.now().strftime("%Y-%m-%d")
#date_str = '2018-04-03'

js = pd.read_json('/Users/taj/GitHub/scraping/stayz/WebData/nsw_calendar/stayz_calendar_' + date_str + '.json.zip')


In [4]:

# No need to change the index, iterating over every row
#js = js.reset_index(drop=True)
#js = js.set_index('property_id')

# Change values of -1.0 into NaN for stats analysis
js.loc[js['review_count'] == -1.0, 'review_count'] = np.nan
js.loc[js['review_value'] == -1.0, 'review_value'] = np.nan

# Remove the properties where property_id = 0000 as these are errors
js2 = js[js['property_id'] != 0]
#js3 = js2[js2['property_id'] != '']


js2 = js.fillna('')

js2.head()

,calendar,ext_at,property_id,review_count,review_value
0,"<thead><tr><th scope=""col""><span title=""Monday...",2018-04-03 01:30:06,4844019,3,5
1,"<thead><tr><th scope=""col""><span title=""Monday...",2018-04-03 01:30:09,9059153,62,5
2,"<thead><tr><th scope=""col""><span title=""Monday...",2018-04-03 01:30:11,9208330,12,4.8
3,"<thead><tr><th scope=""col""><span title=""Monday...",2018-04-03 01:30:14,9024870,89,4.9
4,"<thead><tr><th scope=""col""><span title=""Monday...",2018-04-03 01:30:15,9020763,24,4.9


In [5]:
# Change the index to property id
js3 = js.set_index('property_id')
js3.head()

,calendar,ext_at,review_count,review_value
property_id,,,,
4844019,"<thead><tr><th scope=""col""><span title=""Monday...",2018-04-03 01:30:06,3.0,5.0
9059153,"<thead><tr><th scope=""col""><span title=""Monday...",2018-04-03 01:30:09,62.0,5.0
9208330,"<thead><tr><th scope=""col""><span title=""Monday...",2018-04-03 01:30:11,12.0,4.8
9024870,"<thead><tr><th scope=""col""><span title=""Monday...",2018-04-03 01:30:14,89.0,4.9
9020763,"<thead><tr><th scope=""col""><span title=""Monday...",2018-04-03 01:30:15,24.0,4.9


In [6]:
a = js[js['property_id'] == 9228503]
a

,calendar,ext_at,property_id,review_count,review_value
659,"<thead><tr><th scope=""col""><span title=""Monday...",2018-04-03 01:52:32,9228503,2.0,5.0


In [7]:
d = datetime.date.today()
mr = monthrange(d.year, d.month )

cur_month = d.month
cur_year = d.year


first_page = True



fp = open('/Users/taj/GitHub/scraping/stayz/WebData/nsw_processed_calendar/stayz_processed_calendar_' + date_str + '.json', 'w')


for index, row in js.iterrows():   
    
    all_months = {}
    
    cur_month = d.month
    
    print("")
    
    try:
        raw_cal = "<html><body><table>" + row['calendar'] + "</table></body></html>"
    except TypeError:
        print("Error with the calendar row....")
        raw_cal = "<html><body><table></table></body></html>"
    
    soup = BeautifulSoup(raw_cal,'lxml') # Parse the HTML as a string
    soup_tables = soup.findAll('tbody')# Grab the first table
    
    
    month_tb_count = 1

    for month_tb in soup_tables:
 
        cells = month_tb.findAll("td")
        
        for td in cells:
            try:
                day_val = int(td.text)

                date_full = datetime.date(cur_year,cur_month,day_val)

                m1 = None
                m2 = None
                m3 = None
                m4 = None

                #----------------------------------------------------------
                # Check if DEP and ARR can both be on the same day
                # for consecutive bookings?

                # If its an unavailable date then mark: 
                m1 = re.search('c-calendar--unavailable', str(td))

                if m1 is not None:
                    all_months[str(date_full)] = 'UVL' # Unavailable day

                # If its an unavailable date then mark: 
                m2 = re.search('c-calendar--arrival-day', str(td))

                if m2 is not None:
                    all_months[str(date_full)] = 'ARR' # Arrival day

                # If its an unavailable date then mark: 
                m3 = re.search('c-calendar--departure-day', str(td))

                if m3 is not None:
                    all_months[str(date_full)] = 'DEP' # Departure day

                m4 = re.search('c-calendar--available', str(td))
                if m4 is not None:
                    all_months[str(date_full)] = 'AVL' # Available day


            except ValueError:
                day_val = None

        # Move to the next month
        cur_month += 1
        month_tb_count += 1

    
    cal_details = {}
    cal_details['property_id'] = row['property_id']
    cal_details['calendar'] = all_months
    
    if first_page is True:
        fp.write('[\n')
        first_page = False
    else:
        fp.write('\n,')
        
    # Write the data to JSON
    json.dump(cal_details, fp)

# Show process completed
print("Completed processing....")
    
# Close out the JSON format
fp.write('\n]')    

# Tidy up
fp.close()

In [8]:
# Open the Processed Calendar file
p = pd.read_json('/Users/taj/GitHub/scraping/stayz/WebData/nsw_processed_calendar/stayz_processed_calendar_' + date_str + '.json')
p.head()

,calendar,property_id
0,"{'2018-04-01': 'AVL', '2018-04-02': 'ARR', '20...",4844019
1,"{'2018-04-01': 'UVL', '2018-04-02': 'UVL', '20...",9059153
2,"{'2018-04-01': 'AVL', '2018-04-02': 'ARR', '20...",9208330
3,"{'2018-04-01': 'AVL', '2018-04-02': 'ARR', '20...",9024870
4,"{'2018-04-01': 'UVL', '2018-04-02': 'UVL', '20...",9020763


In [9]:
# Open the file for writing out the bookings details:
fp = open('/Users/taj/GitHub/scraping/stayz/WebData/nsw_bookings/stayz_bookings_' + date_str + '.json', 'w')

first_page = True

# Iterate over each entry in the Processed Calendar file:
for index, row in p.iterrows():  
    
    c = row['calendar']
    
    pid = row['property_id']
    
    days_count = 0
    avl_count = 0
    dep_count = 0
    arr_count = 0
    uvl_count = 0

    booking_count = 0

    dates = list(c.keys())

    min_dateIndex = 0
    max_dateIndex = len(dates)
 
    while (min_dateIndex < max_dateIndex):

        date = dates[min_dateIndex]
        
        #print("Date: " + str(date))
        status = c[date]

        # Count the total number of days
        days_count += 1

        if( status == 'AVL'):
            avl_count += 1

        if( status == 'ARR'):
            arr_count += 1
            # Iterate while the days are Unavailble until we find a Departure
            
            #print("Arrival date: " + str(date))

            # Keep the arrival date:
            date_arr = date

            # Reset the count for this booking
            booking_days = 0

            # Move to the next day.
            # Breaks if they arrive on the last day of the 6th month!!!
            min_dateIndex += 1
            #booking_days += 1
            
            if(min_dateIndex < max_dateIndex): 
                date = dates[min_dateIndex]
                status = c[date]
            else:
                date = None
                status = None
            
            #print("Looping through booking....")
            # If only one night stay. be careful how to increment booking_days???
            while(( status != 'DEP') & (min_dateIndex < max_dateIndex)):  
                #print("Date: {0} - {1}".format(date, status))

                # Departure date doesnt count as a booked date, but as an available date
                booking_days += 1

                days_count += 1

                min_dateIndex += 1
                
                if(min_dateIndex < max_dateIndex): 
                    date = dates[min_dateIndex]
                    status = c[date]
                

            # Add in the last day
            avl_count += 1
            days_count += 1
            booking_days += 1
            
            # Go back a day
            #min_dateIndex -= 1

            # Get the departure day details??
            
            if(min_dateIndex >= max_dateIndex): 
                # Booking runs over the end of the month into the 7th month, which we dont track
                # Ignore this booking or just track to the end of the month?
                date_dep = None
            else:
                date_dep = date

            # Track the total bookings.
            booking_count += booking_days

            # Show the booking details
            booking_detail = {
                'property_id': pid,
                'arr_dt': date_arr,
                'dep_dt': date_dep,
                'book_days': booking_days
            }

            if first_page is True:
                fp.write('[\n')
                first_page = False
            else:
                fp.write('\n,')

            json.dump(booking_detail, fp)        

        if( status == 'UVL'):
            uvl_count += 1

        
        min_dateIndex += 1

# Close off the JSON
fp.write(']')

print("Total days: " + str(days_count))
print("Available days: " + str(avl_count))
print("Departure days: " + str(dep_count))
print("Arrival days: " + str(arr_count))
print("Booked days: " + str(booking_count))
print("Unavailable days: " + str(uvl_count))
print("Total check: " + str(avl_count + booking_count + uvl_count))

# Tidy up file handles
fp.close()

Total days: 183
Available days: 175
Departure days: 0
Arrival days: 2
Booked days: 6
Unavailable days: 1
Total check: 182
